# Convert CD, CB7, and GDCC host-guest pairs from https://github.com/MobleyLab/benchmarksets/ to SMIRNOFF99Frosst

In [111]:
%load_ext autoreload
%autoreload 2

import os as os
import glob as glob
import numpy as np
import pandas as pd

import parmed as pmd

from openforcefield.typing.engines.smirnoff import ForceField, unit
from openforcefield.utils import mergeStructure

from smirnovert.utils import (create_pdb_with_conect, prune_conect, split_topology, create_host_guest_topology,
                    create_host_mol2, convert_mol2_to_sybyl_antechamber,
                    load_mol2, check_unique_atom_names,
                    check_bond_lengths,
                    extract_water_and_ions, create_water_and_ions_parameters)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import logging
from importlib import reload
# `logging` needs to be reloaded, because `jupyter notebook` itself 
# uses the logging module to print messages to standard output...
reload(logging)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p')

## Clone the repository locally (this is easier than looping through all files remotely)

In [12]:
prefix = './benchmarksets/input_files/'
suffix = '/prmtop-rst7/'

In [59]:
benchmarksets = [
    'cb7-set1',
    'cb7-set2',
    'cd-set1',
    'cd-set2',
    'gdcc-set1',
    'gdcc-set2'
]

The two sets of CB7 guests were sutdied experimentally under uniform conditions -- 50 mM sodium acetate buffer, pH 4.74, 298K -- by a single research group.

In [153]:
cd_set1 = pd.read_table(prefix + 'cd-set1' + '/README.md', skiprows=lambda x: x in [0, 1, 2, 4], 
              nrows=23, delimiter='|', usecols=(1, 2, 3, 4, 5), keep_default_na=False, 
              skip_blank_lines=False)
# Drop the break between the two "sets" of guests
cd_set1 = cd_set1.drop(cd_set1.index[8])

In [154]:
cd_set1

,Host,Guest,Guest ID,Guest SMILES,Exp ΔG [<sup>a</sup>](#Rek97)
0,aCD,1-butylammonium,1,CCCC[NH3+],-1.575
1,aCD,1-hexylammonium,2,CCCCCC[NH3+],-3.533
2,aCD,1-octylammonium,3,CCCCCCCC[NH3+],-4.606
3,aCD,cyclopentanol,4,OC1CCCC1,-2.130
4,aCD,cycloheptanol,5,OC1CCCCCC1,-2.510
5,aCD,butanoate,6,CCCC([O-])=O,-1.506
6,aCD,hexanoate,7,CCCCCC([O-])=O,-3.380
7,aCD,octanoate,8,CCCCCCCC([O-])=O,-4.622
9,aCD,n-methylbutylammonium,s9,CCCC[NH2+]C,-1.685
10,aCD,1-methylbutylammonium,s10,CCCC(C)[NH3+],-1.764


In [148]:
# cb7_set1['charge'] = [1, 2, 2, 2, -1, ]

,Host,Guest,Guest ID,Guest SMILES,Exp ΔG [<sup>a</sup>](#Rek97)
0,aCD,1-butylammonium,1,CCCC[NH3+],-1.575
1,aCD,1-hexylammonium,2,CCCCCC[NH3+],-3.533
2,aCD,1-octylammonium,3,CCCCCCCC[NH3+],-4.606
3,aCD,cyclopentanol,4,OC1CCCC1,-2.130
4,aCD,cycloheptanol,5,OC1CCCCCC1,-2.510
5,aCD,butanoate,6,CCCC([O-])=O,-1.506
6,aCD,hexanoate,7,CCCCCC([O-])=O,-3.380
7,aCD,octanoate,8,CCCCCCCC([O-])=O,-4.622
9,aCD,n-methylbutylammonium,s9,CCCC[NH2+]C,-1.685
10,aCD,1-methylbutylammonium,s10,CCCC(C)[NH3+],-1.764


,Host,Guest,Guest ID,Guest SMILES,Exp ΔG [<sup>a</sup>](#Rek97)
0,aCD,1-butylammonium,1,CCCC[NH3+],-1.575
1,aCD,1-hexylammonium,2,CCCCCC[NH3+],-3.533
2,aCD,1-octylammonium,3,CCCCCCCC[NH3+],-4.606
3,aCD,cyclopentanol,4,OC1CCCC1,-2.130
4,aCD,cycloheptanol,5,OC1CCCCCC1,-2.510
5,aCD,butanoate,6,CCCC([O-])=O,-1.506
6,aCD,hexanoate,7,CCCCCC([O-])=O,-3.380
7,aCD,octanoate,8,CCCCCCCC([O-])=O,-4.622
9,aCD,n-methylbutylammonium,s9,CCCC[NH2+]C,-1.685
10,aCD,1-methylbutylammonium,s10,CCCC(C)[NH3+],-1.764


In [121]:
cb7_set2 = pd.read_table(prefix + 'cd-set2' + '/README.md', skiprows=lambda x: x in [0, 1, 2, 4], 
              nrows=22, delimiter='|', usecols=(1, 2, 3, 4, 5), keep_default_na=False, 
              skip_blank_lines=False)

In [167]:
# systems = {}
systems = []

for system in ['cd-set1']:
    for index, file in enumerate(sorted(glob.glob(prefix + system + suffix + '*.prmtop'))):
        systems.append(file)

In [168]:
systems

['./benchmarksets/input_files/cd-set1/prmtop-rst7/acd-1-p.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-1-s.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-2-p.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-2-s.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-3-p.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-3-s.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-4-p.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-4-s.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-5-p.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-5-s.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-6-p.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-6-s.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-7-p.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-7-s.prmtop',
 './benchmarksets/input_files/cd-set1/prmtop-rst7/acd-8-p.prmt

In [164]:
benchmarksets = [
    'cb7-set1',
    'cb7-set2',
    'cd-set1',
    'cd-set2',
    'gdcc-set1',
    'gdcc-set2'
]
prefix = './benchmarksets/input_files/'
suffix = '/prmtop-rst7/'

In [175]:
systems[0][:-7].split('/')[-1]

'acd-1-p'

In [200]:
for file in systems:
    base_name = file[:-7].split('/')[-1]
    destination = prefix + 'cd-set1' + suffix + 'smirnoff/' + base_name + '/'
    reference_prmtop = base_name + '.prmtop'
    reference_inpcrd = base_name + '.rst7'
    host_resname = 'MGO'
    guest_resname = 'MOL'
    
    if not os.path.exists(destination):
        os.makedirs(destination)
        os.symlink('../../' + reference_prmtop, destination + reference_prmtop)
        os.symlink('../../' + reference_inpcrd, destination + reference_inpcrd)
    
    convert(destination, base_name, reference_prmtop, reference_inpcrd, host_resname, guest_resname)

2018-03-07 04:48:51 PM Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/acd-1-p.pdb with CONECT records...
2018-03-07 04:48:51 PM Pruning water-water CONECT records...
2018-03-07 04:48:51 PM Splitting topology into components...
2018-03-07 04:48:52 PM Creating a combined topology for the host and guest molecules...
2018-03-07 04:48:52 PM Writing a `mol2` for the host molecule...
2018-03-07 04:48:52 PM Writing a `mol2` for the host molecule...
2018-03-07 04:48:53 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/MGO.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:48:53 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/MOL.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:48:53 PM Extracting water and ions from acd-1-p.prmtop...
2018-03-07 04:48:53 PM Creating parameters for the waters and ions...
2018-03-07 04:48:54 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-

Check if the host-guest parameter file already exists...
Check if the host-guest coordinate file already exists...


2018-03-07 04:48:55 PM Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-s/acd-1-s.pdb with CONECT records...


Check if solvated parameter file already exists...
Check if solvated coordinate file already exists...


2018-03-07 04:48:55 PM Pruning water-water CONECT records...
2018-03-07 04:48:55 PM Splitting topology into components...
2018-03-07 04:48:56 PM Creating a combined topology for the host and guest molecules...
2018-03-07 04:48:57 PM Writing a `mol2` for the host molecule...
2018-03-07 04:48:57 PM Writing a `mol2` for the host molecule...
2018-03-07 04:48:57 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-s/MGO.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:48:57 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-s/MOL.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:48:57 PM Extracting water and ions from acd-1-s.prmtop...
2018-03-07 04:48:57 PM Creating parameters for the waters and ions...
2018-03-07 04:48:58 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-s/MGO-sybyl.mol2...
2018-03-07 04:48:58 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-s/MOL-sybyl.mol2

...

2018-03-07 04:50:09 PM Writing a `mol2` for the host molecule...
2018-03-07 04:50:09 PM Writing a `mol2` for the host molecule...
2018-03-07 04:50:09 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s10-p/MGO.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:50:09 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s10-p/MOL.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:50:09 PM Extracting water and ions from acd-s10-p.prmtop...
2018-03-07 04:50:09 PM Creating parameters for the waters and ions...
2018-03-07 04:50:10 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s10-p/MGO-sybyl.mol2...
2018-03-07 04:50:10 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s10-p/MOL-sybyl.mol2...
2018-03-07 04:50:10 PM Checking all atoms have unique names...
2018-03-07 04:50:10 PM Checking all atoms have unique names...
2018-03-07 04:50:11 PM Checking structure for bonds >4 A...
2018-03-07

2018-03-07 04:50:44 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s13-p/MOL.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:50:44 PM Extracting water and ions from acd-s13-p.prmtop...
2018-03-07 04:50:44 PM Creating parameters for the waters and ions...
2018-03-07 04:50:45 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s13-p/MGO-sybyl.mol2...
2018-03-07 04:50:45 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s13-p/MOL-sybyl.mol2...
2018-03-07 04:50:45 PM Checking all atoms have unique names...
2018-03-07 04:50:45 PM Checking all atoms have unique names...
2018-03-07 04:50:46 PM Checking structure for bonds >4 A...
2018-03-07 04:50:48 PM Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s13-s/acd-s13-s.pdb with CONECT records...
2018-03-07 04:50:49 PM Pruning water-water CONECT records...
2018-03-07 04:50:49 PM Splitting topology into components...
2018-03-07 04:50:50 PM Creating

2018-03-07 04:51:20 PM Creating parameters for the waters and ions...
2018-03-07 04:51:21 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s16-p/MGO-sybyl.mol2...
2018-03-07 04:51:21 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s16-p/MOL-sybyl.mol2...
2018-03-07 04:51:21 PM Checking all atoms have unique names...
2018-03-07 04:51:21 PM Checking all atoms have unique names...
2018-03-07 04:51:21 PM Checking structure for bonds >4 A...
2018-03-07 04:51:24 PM Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s16-s/acd-s16-s.pdb with CONECT records...
2018-03-07 04:51:24 PM Pruning water-water CONECT records...
2018-03-07 04:51:24 PM Splitting topology into components...
2018-03-07 04:51:25 PM Creating a combined topology for the host and guest molecules...
2018-03-07 04:51:25 PM Writing a `mol2` for the host molecule...
2018-03-07 04:51:25 PM Writing a `mol2` for the host molecule...
2018-03-07 04:51:25 PM Converting ./

2018-03-07 04:51:57 PM Loading ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s19-p/MOL-sybyl.mol2...
2018-03-07 04:51:57 PM Checking all atoms have unique names...
2018-03-07 04:51:57 PM Checking all atoms have unique names...
2018-03-07 04:51:57 PM Checking structure for bonds >4 A...
2018-03-07 04:52:00 PM Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s19-s/acd-s19-s.pdb with CONECT records...
2018-03-07 04:52:00 PM Pruning water-water CONECT records...
2018-03-07 04:52:00 PM Splitting topology into components...
2018-03-07 04:52:01 PM Creating a combined topology for the host and guest molecules...
2018-03-07 04:52:01 PM Writing a `mol2` for the host molecule...
2018-03-07 04:52:01 PM Writing a `mol2` for the host molecule...
2018-03-07 04:52:01 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s19-s/MGO.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:52:01 PM Converting ./benchmarksets/input_files/cd-set1/prm

2018-03-07 04:52:32 PM Checking all atoms have unique names...
2018-03-07 04:52:33 PM Checking structure for bonds >4 A...
2018-03-07 04:52:35 PM Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s22-s/acd-s22-s.pdb with CONECT records...
2018-03-07 04:52:35 PM Pruning water-water CONECT records...
2018-03-07 04:52:35 PM Splitting topology into components...
2018-03-07 04:52:37 PM Creating a combined topology for the host and guest molecules...
2018-03-07 04:52:37 PM Writing a `mol2` for the host molecule...
2018-03-07 04:52:37 PM Writing a `mol2` for the host molecule...
2018-03-07 04:52:37 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s22-s/MGO.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:52:37 PM Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-s22-s/MOL.mol2 to SYBYL atom types via Antechamber...
2018-03-07 04:52:37 PM Extracting water and ions from acd-s22-s.prmtop...
2018-03-07 04:52:37 PM Creati

In [199]:
def convert(destination, prefix, reference_prmtop, reference_inpcrd, host_resname, guest_resname):
    
        
    reference = pmd.load_file(destination + reference_prmtop, xyz=destination + reference_inpcrd)
    box = reference.box
    
    create_pdb_with_conect(solvated_pdb=destination + reference_inpcrd,
                      amber_prmtop=destination + reference_prmtop,
                      output_pdb=destination + prefix + '.pdb')
    
    prune_conect(input_pdb=prefix + '.pdb',
            output_pdb=prefix + '.pruned.pdb',
            path=destination)
    
    components = split_topology(file_name=destination + prefix + '.pruned.pdb')
    hg_topology = create_host_guest_topology(components, 
                                         host_resname=host_resname, 
                                         guest_resname=guest_resname)
    
    create_host_mol2(
        solvated_pdb=destination + prefix + '.pruned.pdb',
        amber_prmtop=destination + reference_prmtop,
        mask=host_resname,
        output_mol2=destination + host_resname + '.mol2')

    create_host_mol2(
        solvated_pdb=destination + prefix + '.pdb',
        amber_prmtop=destination + reference_prmtop,
        mask=guest_resname,
        output_mol2=destination + guest_resname + '.mol2')

    convert_mol2_to_sybyl_antechamber(
        input_mol2=destination + host_resname + '.mol2',
        output_mol2=destination + host_resname + '-sybyl.mol2',
        ac_doctor=False)

    convert_mol2_to_sybyl_antechamber(
        input_mol2=destination + guest_resname + '.mol2',
        output_mol2=destination + guest_resname + '-sybyl.mol2',
        ac_doctor=False)
    
    extract_water_and_ions(
        amber_prmtop=reference_prmtop,
        amber_inpcrd=reference_inpcrd,
        host_residue=':' + host_resname,
        guest_residue=':' + guest_resname,
        dummy_atoms=True,
        output_pdb='water_ions.pdb',
        path=destination)

    create_water_and_ions_parameters(
        input_pdb='water_ions.pdb',
        output_prmtop='water_ions.prmtop',
        output_inpcrd='water_ions.inpcrd',
        dummy_atoms=False,
        path=destination)
    
    host = load_mol2(
        filename=destination + host_resname + '-sybyl.mol2',
        name=host_resname,
        add_tripos=True)

    guest = load_mol2(
        filename=destination + guest_resname + '-sybyl.mol2',
        name=guest_resname,
        add_tripos=False)

    check_unique_atom_names(host)
    check_unique_atom_names(guest)
    molecules = [host, guest]
    
    ff = ForceField('forcefield/smirnoff99Frosst.ffxml')
    system = ff.createSystem(
        hg_topology.topology,
        molecules,
        nonbondedCutoff=1.1 * unit.nanometer,
        ewaldErrorTolerance=1e-4)
    
    hg_structure = pmd.openmm.topsystem.load_topology(
    hg_topology.topology, system, hg_topology.positions)

    check_bond_lengths(hg_structure, threshold=4)

    try:
        hg_structure.save(destination + 'hg.prmtop')
    except OSError:
        print(
            'Check if the host-guest parameter file already exists...')

    try:
        hg_structure.save(destination + 'hg.inpcrd')
    except OSError:
        print(
            'Check if the host-guest coordinate file already exists...')
        
    water_and_ions = pmd.amber.AmberParm(
    destination + 'water_ions.prmtop',
    xyz=destination + 'water_ions.inpcrd')
    
    merged = mergeStructure(hg_structure, water_and_ions)
    merged.box = reference.box
    try:
        merged.save(destination + 'smirnoff.prmtop')
    except:
        print('Check if solvated parameter file already exists...')
    try:
        merged.save(destination + 'smirnoff.inpcrd')
    except:
        print('Check if solvated coordinate file already exists...')